
<h1> <center>IBM Applied Data Science Capstone Course by Coursera</center></h1>
<h1> <center>Week 5 Final Report</center></h1>
<h1><center>Opening a New Bakery in Dubai, UAE</center></h1>

- __Build a dataframe of neighborhoods in Dubai, UAE by web scraping the data from Wikipedia page__ 
- __Get the geographical coordinates of the neighborhoods__
- __Obtain the venue data for the neighborhoods from Foursquare API__
- __Explore and cluster the neighborhoods__
- __Select the best cluster to open a new Bakery__

In [130]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
 # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [91]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_communities_in_Dubai").text

In [92]:

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'lxml')
table = soup.find('table')
rowsList = []
for rows in table.find_all('tr'):
    rowsList.append(rows.text)

In [93]:
rowsList

['\nCommunity Number\n\nCommunity (English)\nCommunity (Arabic)\nArea(km2)\nPopulation(2000)\nPopulation density(/km2)\n',
 '\n126\n\nAbu Hail\nأبو هيل\n1.27 km²\n21,414\n16,861.4/km²\n',
 '\n711\n\nAl Awir First\nالعوير الأولى\n\n\n\n',
 '\n721\n\nAl Awir Second\nالعوير الثانية\n\n\n\n',
 '\n333\n\nAl Bada\nالبدع\n0.82 km²\n18,816\n22946/km²\n',
 '\n122\n\nAl Baraha\nالبراحة\n1.104 km²\n7,823\n7,086/km²\n',
 '\n373\n\nAl Barsha First\nالبرشاء الأولى\n\n\n\n',
 '\n376\n\nAl Barsha Second\nالبرشاء الثانية\n\n\n\n',
 '\n671\n\nAl Barsha South First\nالبرشاء جنوب الاولى\n\n\n\n',
 '\n672\n\nAl Barsha South Second\nالبرشاء جنوب الثانية\n\n\n\n',
 '\n673\n\nAl Barsha South Third\nالبرشاء جنوب الثالثة\n\n\n\n',
 '\n375\n\nAl Barsha Third\nالبرشاء الثالثة\n\n\n\n',
 '\n114\n\nAl Buteen\nالبطين\n0.07 km²\n2,364\n33,771/km²\n',
 '\n113\n\nAl Dhagaya\nالضغاية\n0.125 km²\n10,896\n21,451/km²\n',
 '\n214\n\nAl Garhoud\nالقرهود\n4 km²\n4,466\n1,116.5/km²\n',
 '\n359\n\nAl Guoz Fourth\nالقوز الرابعة\

In [94]:
CommNum = []
CommName = []
Area = []
Neighborhood_data = {'CommNum':[], 'CommName':[], 'Area':[]} # neighborhood data
# rowslist includes '\n'. 
# Split using '\n' and extract the postcode, Borough and Neighborhood text for every row.
for row in rowsList:
    CommNum.append(row.split('\n')[1])
    CommName.append(row.split('\n')[3])
    Area.append(row.split('\n')[5])

# Fill dictionary    
Neighborhood_data['CommNum'] = CommNum[1:]
Neighborhood_data['CommName'] = CommName[1:]
Neighborhood_data['Area'] = Area[1:]

# Make dataframe of neighborhood data using dictionary
Neighbors = pd.DataFrame(data = Neighborhood_data)


In [95]:
Neighbors.columns=['CommunityNumber', 'CommunityName', 'Area']
Neighbors_orig = Neighbors
Neighbors.head()

,CommunityNumber,CommunityName,Area
0,126,Abu Hail,1.27 km²
1,711,Al Awir First,
2,721,Al Awir Second,
3,333,Al Bada,0.82 km²
4,122,Al Baraha,1.104 km²


In [96]:
Neighbors.shape

(130, 3)

In [97]:
!pip install geocoder
import geocoder

In [98]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Dubai, UAE'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [99]:
coords = [ get_latlng(neighborhood) for neighborhood in Neighbors["CommunityName"].tolist() ]

In [100]:
coords

[[25.28308000000004, 55.33435000000003],
 [25.18593000000004, 55.54126000000008],
 [25.167920000000038, 55.543310000000076],
 [25.21861000000007, 55.26406000000003],
 [25.282800000000066, 55.31678000000005],
 [25.11483000000004, 55.19136000000003],
 [25.107230000000072, 55.20485000000008],
 [25.08958000000007, 55.23424000000006],
 [25.077390000000037, 55.24267000000003],
 [25.062290000000075, 55.23995000000008],
 [25.093420000000037, 55.19044000000008],
 [25.269250000000056, 55.29944000000006],
 [25.272170000000074, 55.30157000000003],
 [25.243370000000027, 55.352670000000046],
 [25.269510000000025, 55.30884000000003],
 [25.25696000000005, 55.30246000000005],
 [25.29871000000003, 55.33546000000007],
 [25.237130000000036, 55.27707000000004],
 [25.220540000000028, 55.34166000000005],
 [25.233420000000024, 55.29001000000005],
 [25.245290000000068, 55.30364000000003],
 [25.27177000000006, 55.33762000000007],
 [25.24282000000005, 55.48440000000005],
 [25.22784000000007, 55.522320000000036],

In [101]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [102]:
Neighbors['Latitude'] = df_coords['Latitude']
Neighbors['Longitude'] = df_coords['Longitude']

In [103]:
print(Neighbors.shape)
Neighbors

(130, 5)


,CommunityNumber,CommunityName,Area,Latitude,Longitude
0,126,Abu Hail,1.27 km²,25.283080,55.334350
1,711,Al Awir First,,25.185930,55.541260
2,721,Al Awir Second,,25.167920,55.543310
3,333,Al Bada,0.82 km²,25.218610,55.264060
4,122,Al Baraha,1.104 km²,25.282800,55.316780
5,373,Al Barsha First,,25.114830,55.191360
6,376,Al Barsha Second,,25.107230,55.204850
7,671,Al Barsha South First,,25.089580,55.234240
8,672,Al Barsha South Second,,25.077390,55.242670
9,673,Al Barsha South Third,,25.062290,55.239950


In [104]:
Neighbors.to_csv("Neighbors.csv", index=False)

In [105]:
# get the coordinates of Dubai
address = 'Dubai, UAE'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dubai, UAE {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dubai, UAE 25.0657, 55.1713.


In [106]:
map_dxb = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(Neighbors['Latitude'], Neighbors['Longitude'], Neighbors['CommunityName']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dxb)  
map_dxb

In [107]:
# define Foursquare Credentials and Version
CLIENT_ID = 'JLGDBXXXFESRV4SV1YKJU5JJ2PS41HZOTPJ4LYUKSSTHO3YI' # your Foursquare ID
CLIENT_SECRET = 'DWEWKD3YKLSAFTUPBMDBWQ05EUN1CO5CYGS5T0QPCI4SVCUU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JLGDBXXXFESRV4SV1YKJU5JJ2PS41HZOTPJ4LYUKSSTHO3YI
CLIENT_SECRET:DWEWKD3YKLSAFTUPBMDBWQ05EUN1CO5CYGS5T0QPCI4SVCUU


In [109]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(Neighbors['Latitude'], Neighbors['Longitude'], Neighbors['CommunityName']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [110]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['CommunityName', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(8770, 7)


,CommunityName,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abu Hail,25.28308,55.33435,Habib Bakery,25.281124,55.332774,Bakery
1,Abu Hail,25.28308,55.33435,Gold's Gym,25.282698,55.341019,Gym
2,Abu Hail,25.28308,55.33435,Al Douri Roastery,25.277057,55.328223,Bakery
3,Abu Hail,25.28308,55.33435,Union Co-Operative Society,25.282769,55.340896,Department Store
4,Abu Hail,25.28308,55.33435,Fitness Time (وقت اللياقة),25.289077,55.347913,Gym


In [111]:
venues_df.groupby(["CommunityName"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
CommunityName,,,,,,
Abu Hail,100,100,100,100,100,100
Al Awir First,8,8,8,8,8,8
Al Awir Second,9,9,9,9,9,9
Al Bada,100,100,100,100,100,100
Al Baraha,100,100,100,100,100,100
Al Barsha First,100,100,100,100,100,100
Al Barsha Second,100,100,100,100,100,100
Al Barsha South First,23,23,23,23,23,23
Al Barsha South Second,23,23,23,23,23,23


In [112]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 313 uniques categories.


In [113]:
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Gym', 'Department Store', 'Market',
       'Performing Arts Venue', 'Fast Food Restaurant',
       'Seafood Restaurant', 'Hotel', 'Indian Restaurant',
       'Mediterranean Restaurant', 'Restaurant', 'Iraqi Restaurant',
       'Fried Chicken Joint', 'Café', 'Lounge', 'Ice Cream Shop',
       'Middle Eastern Restaurant', 'BBQ Joint', 'American Restaurant',
       'Asian Restaurant', 'Dessert Shop', 'Bowling Alley',
       'Burger Joint', 'Burrito Place', 'Nightclub', 'Pool Hall',
       'Comedy Club', 'Pizza Place', 'Fishing Store',
       'Bavarian Restaurant', 'Gym / Fitness Center', 'Convenience Store',
       'Italian Restaurant', 'Tea Room', 'Thai Restaurant', 'Sports Bar',
       'Buffet', 'Moroccan Restaurant', 'Smoke Shop', 'Hotel Bar',
       'Chinese Restaurant', 'Filipino Restaurant', 'Food Court',
       'Soccer Field', 'Hookah Bar', 'Arepa Restaurant', 'Coffee Shop',
       'Scenic Lookout', 'Bar', 'Gas Station'], dtype=object)

In [114]:
"Bakery" in venues_df['VenueCategory'].unique()

True

In [115]:
# one hot encoding
db_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
db_onehot['CommunityNames'] = venues_df['CommunityName'] 

# move neighborhood column to the first column
fixed_columns = [db_onehot.columns[-1]] + list(db_onehot.columns[:-1])
db_onehot = db_onehot[fixed_columns]

print(db_onehot.shape)
db_onehot.head()

(8770, 314)


,CommunityNames,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Belgian Restaurant,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Camera Store,Campground,Canal,Candy Store,Carpet Store,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Doner Restaurant,Donut Shop,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighting Store,Lingerie Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Organic Grocery,Pakistani Restaurant,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Bathroom,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Syri

In [116]:
db_grouped = db_onehot.groupby(["CommunityNames"]).mean().reset_index()

print(db_grouped.shape)
db_grouped

(130, 314)


,CommunityNames,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Belgian Restaurant,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Camera Store,Campground,Canal,Candy Store,Carpet Store,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Doner Restaurant,Donut Shop,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighting Store,Lingerie Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Organic Grocery,Pakistani Restaurant,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Bathroom,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Syri

In [117]:
len(db_grouped[db_grouped["Bakery"] > 0])

70

In [118]:
db_mall = db_grouped[["CommunityNames","Bakery"]]

In [119]:
db_mall.head()

,CommunityNames,Bakery
0,Abu Hail,0.04
1,Al Awir First,0.00
2,Al Awir Second,0.00
3,Al Bada,0.01
4,Al Baraha,0.02


In [120]:
# set number of clusters
kclusters = 3

db_clustering = db_mall.drop(["CommunityNames"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(db_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 0, 1, 0, 1, 2, 2, 0], dtype=int32)

In [121]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
db_merged = db_mall.copy()

# add clustering labels
db_merged["Cluster Labels"] = kmeans.labels_

In [122]:
db_merged.rename(columns={"CommunityNames": "CommunityName"}, inplace=True)
db_merged.head()

,CommunityName,Bakery,Cluster Labels
0,Abu Hail,0.04,1
1,Al Awir First,0.00,0
2,Al Awir Second,0.00,0
3,Al Bada,0.01,0
4,Al Baraha,0.02,1


In [123]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
db_merged = db_merged.join(Neighbors.set_index("CommunityName"), on="CommunityName")

print(db_merged.shape)
db_merged.head() # check the last columns!

(130, 7)


,CommunityName,Bakery,Cluster Labels,CommunityNumber,Area,Latitude,Longitude
0,Abu Hail,0.04,1,126,1.27 km²,25.28308,55.33435
1,Al Awir First,0.00,0,711,,25.18593,55.54126
2,Al Awir Second,0.00,0,721,,25.16792,55.54331
3,Al Bada,0.01,0,333,0.82 km²,25.21861,55.26406
4,Al Baraha,0.02,1,122,1.104 km²,25.28280,55.31678


In [124]:
# sort the results by Cluster Labels
print(db_merged.shape)
db_merged.sort_values(["Cluster Labels"], inplace=True)
db_merged


(130, 7)


,CommunityName,Bakery,Cluster Labels,CommunityNumber,Area,Latitude,Longitude
64,Al Shindagha,0.000000,0,311,0.26 km²,25.269900,55.289840
109,Palm Jumeira,0.000000,0,381,,25.111573,55.140086
65,Al Souq Al Kabeer,0.000000,0,312,,25.260040,55.294980
128,Za'abeel First,0.000000,0,325,10 km²,25.225760,55.299370
63,Al Satwa,0.010000,0,334,2.87 km²,25.225790,55.280790
62,Al Safouh Second,0.000000,0,382,7.6 km²,25.094130,55.156430
61,Al Safouh First,0.010000,0,372,7.6 km²,25.113690,55.175470
60,Al Safa Second,0.000000,0,357,4.5 km²,25.166330,55.231830
59,Al Safa First,0.010000,0,353,4.5 km²,25.177930,55.234220
58,Al Sabkha,0.000000,0,115,0.13 km²,25.268950,55.302570


In [131]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(db_merged['Latitude'], db_merged['Longitude'], db_merged['CommunityName'], db_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [132]:
db_merged.loc[db_merged['Cluster Labels'] == 0]

,CommunityName,Bakery,Cluster Labels,CommunityNumber,Area,Latitude,Longitude
64,Al Shindagha,0.000000,0,311,0.26 km²,25.269900,55.289840
109,Palm Jumeira,0.000000,0,381,,25.111573,55.140086
65,Al Souq Al Kabeer,0.000000,0,312,,25.260040,55.294980
128,Za'abeel First,0.000000,0,325,10 km²,25.225760,55.299370
63,Al Satwa,0.010000,0,334,2.87 km²,25.225790,55.280790
62,Al Safouh Second,0.000000,0,382,7.6 km²,25.094130,55.156430
61,Al Safouh First,0.010000,0,372,7.6 km²,25.113690,55.175470
60,Al Safa Second,0.000000,0,357,4.5 km²,25.166330,55.231830
59,Al Safa First,0.010000,0,353,4.5 km²,25.177930,55.234220
58,Al Sabkha,0.000000,0,115,0.13 km²,25.268950,55.302570


In [133]:
db_merged.loc[db_merged['Cluster Labels'] == 1]

,CommunityName,Bakery,Cluster Labels,CommunityNumber,Area,Latitude,Longitude
108,Oud Al Muteena Third,0.055556,1,268,,25.27326,55.44738
123,Umm Suqeim Second,0.040000,1,362,7.2 km²,25.15259,55.20044
103,Nad Shamma,0.018519,1,213,1.36 km²,25.22128,55.37933
102,Nad Al Hammar,0.023256,1,416,8.4 km²,25.20099,55.37640
97,Mirdif,0.040816,1,251,9.2 km²,25.22614,55.42421
121,Umm Ramool,0.026316,1,215,3.61 km²,25.23159,55.37724
122,Umm Suqeim First,0.021053,1,356,7.2 km²,25.16233,55.21561
0,Abu Hail,0.040000,1,126,1.27 km²,25.28308,55.33435
93,Jumeira First,0.030000,1,332,6.9 km2,25.21054,55.25265
4,Al Baraha,0.020000,1,122,1.104 km²,25.28280,55.31678


In [134]:
db_merged.loc[db_merged['Cluster Labels'] == 2]

,CommunityName,Bakery,Cluster Labels,CommunityNumber,Area,Latitude,Longitude
77,Arabian Ranches,0.080000,2,600,,25.05765,55.27140
31,Al Mizhar Second,0.071429,2,263,11.2 km²,25.25236,55.45677
8,Al Barsha South Second,0.086957,2,672,,25.07739,55.24267
7,Al Barsha South First,0.086957,2,671,,25.08958,55.23424
125,Wadi Alamardi,0.100000,2,271,,25.21703,55.48636
99,Muhaisanah Second,0.125000,2,264,13 km²,25.26488,55.42487
22,Al Khwaneej First,0.066667,2,281,,25.24282,55.48440


# Conclusion

Most of the Bakeries are concentrated in the central area of Dubai city, with the highest number in cluster 0 and moderate number in cluster 1. On the other hand, cluster 2 has very low number to totally no Bakery in the neighborhoods. This represents a great opportunity and high potential areas to open new Bakery as there is very little to no competition from Bakeries. Meanwhile, Bakeries in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of Bakeries. From another perspective, this also shows that the oversupply of Bakeries mostly happened in the central area of the city, with the suburb area still have very few Bakeries. Therefore, this project recommends property developers to capitalize on these findings to open new Bakeries in neighborhoods in cluster 2 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.